In [94]:
import cv2
import numpy as np
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import time
import os
import json
import shutil

In [29]:
def timing_decorator(func):
    def wrapper(*args, **kwargs):
        start_time = time.time()
        result = func(*args, **kwargs)
        end_time = time.time()
        print(f"{func.__name__} execution time: {end_time - start_time} seconds")
        return result
    return wrapper

In [99]:
def crop_center(image, crop_fraction=0.8, max_total_pixels = 4000):
    h, w, _ = image.shape
    crop_h, crop_w = int(h * crop_fraction), int(w * crop_fraction)
    start_h, start_w = (h - crop_h) // 2, (w - crop_w) // 2
    cropped_image = image[start_h:start_h + crop_h, start_w:start_w + crop_w]

    # Check if resizing is needed
    total_pixels = cropped_image.shape[0] * cropped_image.shape[1]
    if total_pixels > max_total_pixels:
        scale_factor = (max_total_pixels / total_pixels) ** 0.5
        new_h, new_w = int(cropped_image.shape[0] * scale_factor), int(cropped_image.shape[1] * scale_factor)
        cropped_image = cv2.resize(cropped_image, (new_w, new_h), interpolation=cv2.INTER_AREA)
    
    return cropped_image

@timing_decorator
def find_top_colors_hsv(image_path, k=4):
    # Load image
    image = cv2.imread(image_path)
    # Crop the center 80% of the image
    cropped_image = crop_center(image)
    # Convert image from BGR to HSV
    image_hsv = cv2.cvtColor(cropped_image, cv2.COLOR_BGR2HSV)

    # Perform histogram equalization on the V channel
    # image_hsv[:, :, 2] = cv2.equalizeHist(image_hsv[:, :, 2])
    
    # Reshape image to be a list of pixels
    pixels_hsv = image_hsv.reshape((-1, 3))
    
    # Cluster pixels using KMeans
    kmeans = KMeans(n_clusters=k)
    kmeans.fit(pixels_hsv)
    
    # Get the cluster centers (dominant colors)
    colors_hsv = kmeans.cluster_centers_
    
    # Get the labels of each pixel
    labels = kmeans.labels_
    
    # Replace each pixel with the color of its cluster
    segmented_image_hsv = colors_hsv[labels].reshape(image_hsv.shape)
    
    # Convert the segmented image back to RGB
    segmented_image_rgb = cv2.cvtColor(segmented_image_hsv.astype(np.uint8), cv2.COLOR_HSV2RGB)
    return cropped_image, segmented_image_rgb, colors_hsv, labels, kmeans.cluster_centers_

def plot_segmented_image(segmented_image_rgb):
    plt.figure(figsize=(10, 10))
    plt.imshow(segmented_image_rgb)
    plt.axis('off')
    plt.show()

def plot_colors(colors, counts, cluster_size):
    # Plot each color with its count
    fig, ax = plt.subplots(1, cluster_size, figsize=(15, 5))
    for i, (color, count) in enumerate(zip(colors, counts)):
        color_image = np.zeros((100, 100, 3), dtype=np.uint8)
        color_image[:, :] = color.astype(np.uint8)
        ax[i].imshow(color_image)
        ax[i].set_title(f'Color {i+1}\nCount: {count}')
        ax[i].axis('off')
    plt.show()

def save_color_centers_json(colors_hsv, counts, save_path):
    data = [{"color": color.tolist(), "count": int(count)} for color, count in zip(colors_hsv, counts)]
    with open(save_path, 'w') as f:
        json.dump(data, f)

def create_directory(path):
    if not os.path.exists(path):
        os.makedirs(path)


In [110]:
# Directory to browse through
from tqdm import tqdm
base_directory = 'ColoursRJ'

# Iterate over each color directory
for color in tqdm(os.listdir(base_directory)):
    color_path = os.path.join(base_directory, color)
    
    if os.path.isdir(color_path):
        # Iterate over each image in the color directory
        for image_name in tqdm(os.listdir(color_path)):
            image_path = os.path.join(color_path, image_name)
            
            if os.path.isfile(image_path):
                try:
                    # Process the image
                    cluster_size = 4
                    cropped_image, segmented_image, top_colors_hsv, labels, cluster_centers = find_top_colors_hsv(image_path, k=cluster_size)
                    top_colors_rgb = [cv2.cvtColor(np.uint8([[color]]), cv2.COLOR_HSV2RGB)[0][0] for color in top_colors_hsv]
                    top_counts = np.unique(labels, return_counts=True)[1]

                    # print(f'Processing image: {image_path}')
                    print(f'Top colors: {top_colors_rgb}')
                    print(f'Top counts: {top_counts}')

                    # Plot the top colors
                    # plot_colors(top_colors_rgb, top_counts, cluster_size)

                    # Plot the segmented image
                    # plot_segmented_image(segmented_image)

                    # Prepare to save results
                    base_path = f'labeling/{color}/{os.path.splitext(image_name)[0]}'
                    create_directory(base_path)

                    # Save the original image
                    shutil.copy(image_path, os.path.join(base_path, 'original.png'))

                    # Save each cluster segment
                    for i in range(cluster_size):
                        cluster_mask = (labels == i).reshape(segmented_image.shape[:2])
                        cluster_image = np.zeros_like(cropped_image)
                        cluster_image[cluster_mask] = cropped_image[cluster_mask]
                        cv2.imwrite(os.path.join(base_path, f'color_center_{i+1}.png'), cluster_image)

                    # Save the color centers JSON
                    save_color_centers_json(top_colors_hsv, top_counts, os.path.join(base_path, 'color_centers.json'))
                
                except Exception as e:
                    print(f"Failed to process image {image_path}: {e}")

  0%|          | 0/9 [00:00<?, ?it/s]

find_top_colors_hsv execution time: 0.294780969619751 seconds
Top colors: [array([135, 152,  80], dtype=uint8), array([50, 62, 58], dtype=uint8), array([24, 56, 42], dtype=uint8), array([132, 144, 138], dtype=uint8)]
Top counts: [ 331 1870 1019  740]
find_top_colors_hsv execution time: 0.09239077568054199 seconds
Top colors: [array([149, 143,  61], dtype=uint8), array([56, 72, 54], dtype=uint8), array([19, 69, 46], dtype=uint8), array([37, 74, 54], dtype=uint8)]
Top counts: [ 382  841 1520 1193]
find_top_colors_hsv execution time: 0.08305692672729492 seconds
Top colors: [array([48, 67, 66], dtype=uint8), array([ 99, 143, 144], dtype=uint8), array([ 3, 32, 30], dtype=uint8), array([40, 89, 82], dtype=uint8)]
Top counts: [1019  957  234 1122]


find_top_colors_hsv execution time: 0.11839795112609863 seconds
Top colors: [array([131, 168, 145], dtype=uint8), array([25, 79, 57], dtype=uint8), array([68, 93, 77], dtype=uint8), array([50, 96, 76], dtype=uint8)]
Top counts: [ 597 1338  621 1357]
find_top_colors_hsv execution time: 0.11292910575866699 seconds
Top colors: [array([154, 203, 152], dtype=uint8), array([ 46, 110,  89], dtype=uint8), array([ 3, 40, 21], dtype=uint8), array([ 71, 104,  84], dtype=uint8)]
Top counts: [381 815 344 404]


find_top_colors_hsv execution time: 0.11955094337463379 seconds
Top colors: [array([14, 49, 44], dtype=uint8), array([ 92, 133, 136], dtype=uint8), array([44, 64, 64], dtype=uint8), array([ 44, 109,  98], dtype=uint8)]
Top counts: [ 419 1244 1359  962]
find_top_colors_hsv execution time: 0.11231780052185059 seconds
Top colors: [array([69, 79, 71], dtype=uint8), array([17, 51, 26], dtype=uint8), array([165, 150, 100], dtype=uint8), array([34, 38, 45], dtype=uint8)]
Top counts: [ 999  822  276 1695]


find_top_colors_hsv execution time: 0.14695215225219727 seconds
Top colors: [array([51, 70, 69], dtype=uint8), array([ 72, 118, 125], dtype=uint8), array([124, 140, 136], dtype=uint8), array([30, 66, 60], dtype=uint8)]
Top counts: [1264  952  702 1024]
find_top_colors_hsv execution time: 0.12879610061645508 seconds
Top colors: [array([141, 151, 152], dtype=uint8), array([38, 41, 47], dtype=uint8), array([80, 81, 71], dtype=uint8), array([24, 50, 43], dtype=uint8)]
Top counts: [ 190 1143  263  516]


...

find_top_colors_hsv execution time: 0.05559515953063965 seconds
Top colors: [array([95, 94, 99], dtype=uint8), array([3, 1, 6], dtype=uint8), array([210, 208, 212], dtype=uint8), array([35, 32, 38], dtype=uint8)]
Top counts: [222  34  82 430]
find_top_colors_hsv execution time: 0.1797189712524414 seconds
Top colors: [array([75, 79, 86], dtype=uint8), array([35, 39, 46], dtype=uint8), array([140, 145, 151], dtype=uint8), array([26, 32, 44], dtype=uint8)]
Top counts: [ 851 1413  576 1120]


find_top_colors_hsv execution time: 0.10784697532653809 seconds
Top colors: [array([33, 34, 37], dtype=uint8), array([187, 189, 191], dtype=uint8), array([0, 1, 3], dtype=uint8), array([94, 96, 98], dtype=uint8)]
Top counts: [497 207  65 456]
find_top_colors_hsv execution time: 0.10984396934509277 seconds
Top colors: [array([89, 92, 99], dtype=uint8), array([156, 154, 149], dtype=uint8), array([44, 51, 60], dtype=uint8), array([171, 174, 180], dtype=uint8)]
Top counts: [1341  379 1644  536]
find_top_colors_hsv execution time: 0.061347007751464844 seconds
Top colors: [array([12, 17, 25], dtype=uint8), array([125, 130, 128], dtype=uint8), array([51, 55, 57], dtype=uint8), array([192, 198, 202], dtype=uint8)]
Top counts: [ 88 246 372  94]


find_top_colors_hsv execution time: 0.10689425468444824 seconds
Top colors: [array([32, 34, 42], dtype=uint8), array([67, 71, 75], dtype=uint8), array([161, 165, 169], dtype=uint8), array([ 4,  5, 14], dtype=uint8)]
Top counts: [721 577 244  90]
find_top_colors_hsv execution time: 0.08632421493530273 seconds
Top colors: [array([73, 75, 81], dtype=uint8), array([157, 161, 165], dtype=uint8), array([32, 35, 41], dtype=uint8), array([ 3,  5, 12], dtype=uint8)]
Top counts: [511 312 390  47]
find_top_colors_hsv execution time: 0.04768729209899902 seconds
Top colors: [array([ 5,  7, 13], dtype=uint8), array([39, 39, 45], dtype=uint8), array([153, 157, 158], dtype=uint8), array([46, 45, 44], dtype=uint8)]
Top counts: [ 57 539 205 119]


100%|██████████| 9/9 [00:34<00:00,  3.83s/it]

find_top_colors_hsv execution time: 0.07412409782409668 seconds
Top colors: [array([ 98, 105, 107], dtype=uint8), array([31, 40, 42], dtype=uint8), array([193, 200, 201], dtype=uint8), array([1, 6, 9], dtype=uint8)]
Top counts: [567 459 285 115]
find_top_colors_hsv execution time: 0.07572197914123535 seconds
Top colors: [array([16, 20, 23], dtype=uint8), array([53, 57, 59], dtype=uint8), array([167, 170, 172], dtype=uint8), array([0, 3, 5], dtype=uint8)]
Top counts: [248 640 335  97]
